# Routing

https://developer.here.com/documentation/routing/topics/what-is.html

## Car route from A to B

In [ ]:
from ipyrest import Api
import utils
from credentials import APP_CODE, APP_ID

In [ ]:
start = utils.latlon_for_address('Invalidenstr. 116, 10115 Berlin, Germany')
end = utils.latlon_for_address('Bismarkstr. 35, 10627 Berlin, Germany')

In [ ]:
url = 'https://route.cit.api.here.com/routing/7.2/calculateroute.json'
lat0, lon0 = start
lat1, lon1 = end
params = dict(
    waypoint0=f'geo!{lat0},{lon0}',
    waypoint1=f'geo!{lat1},{lon1}',
    metricsystem='metric',
    jsonattributes='41',
    language='en',
    routeattributes='sh,gr',
    instructionFormat='text',
    mode='fastest;car;traffic:disabled',
    app_id=APP_ID,
    app_code=APP_CODE
)
api = Api(url, params=params)
api

In [ ]:
route = utils.get_route_positions(start, end)
route

In [ ]:
from ipyleaflet import Map, Marker, basemap_to_tiles

center = utils.mid_point(start, end)
here_basemap = utils.build_here_basemap()
layers = [basemap_to_tiles(here_basemap)]
m = Map(center=center, layers=layers, zoom=13)
m

In [ ]:
start_marker = Marker(location=start)
m += start_marker

In [ ]:
end_marker = Marker(location=end)
m += end_marker

In [ ]:
m -= start_marker
m -= end_marker

In [ ]:
utils.add_route_to_map(route, m)

## Time-based isoline with start as center

In [ ]:
from ipyleaflet import Map, Marker
import requests
from ipyleaflet import Map

In [ ]:
lat, lon = 52.5, 13.4
url = (
    'https://isoline.route.api.here.com/routing/7.2/calculateisoline.json'
   f'?app_id={APP_ID}&app_code={APP_CODE}' 
   f'&start=geo!{lat},{lon}'
    '&mode=fastest;car;traffic:disabled'
    '&range=300,600'  # seconds/meters
    '&rangetype=time' # time/distance
    #'&departure=now' # 2018-07-04T17:00:00+02
    #'&resolution=20' # meters
)
obj = requests.get(url).json()

In [ ]:
obj

In [ ]:
from ipyleaflet import Map, Polyline, basemap_to_tiles
import utils

here_basemap = utils.build_here_basemap()
layers = [basemap_to_tiles(here_basemap)]
m = Map(center=(lat, lon), layers=layers, zoom=12)
m

In [ ]:
m += Marker(location=(lat, lon))

In [ ]:
for isoline in obj['response']['isoline']:
    shape = isoline['component'][0]['shape']
    path = [tuple(map(float, pos.split(','))) for pos in shape]
    m += Polyline(locations=path, color='red', weight=2, fill=True)

**Note:** Many more variants! Check https://developer.here.com/api-explorer/rest/routing.